In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import yaml
import os
import pyodbc
import copy
import pandas as pd
import pyodbc
import os
import glob
import numpy as np

# Downloading The Data

In [2]:
# This is the 2021_01 Data
# specifying the path to csv files
path = "C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-56 Estimates 2021 QC/Data/2021_01/Raw Data"
  
# csv files in the path
files = glob.glob(path + "/*.csv")

new = {}
for path in files:
    df = pd.read_csv(path)
    if 'Unnamed: 0' in df.columns:
        df = df.drop('Unnamed: 0', axis = 1)
        print('Has')
    new[path.split('\\')[1][:-4]] = df

In [3]:
new.keys()

dict_keys(['2021_01_age_cpa_QA', '2021_01_age_ethnicity_cpa_QA', '2021_01_age_ethnicity_jurisdiction_QA', '2021_01_age_ethnicity_region_QA', '2021_01_age_jurisdiction_QA', '2021_01_age_region_QA', '2021_01_age_sex_ethnicity_cpa_QA', '2021_01_age_sex_ethnicity_jurisdiction_QA', '2021_01_age_sex_ethnicity_region_QA', '2021_01_consolidated_cpa_QA', '2021_01_consolidated_jurisdiction_QA', '2021_01_consolidated_region_QA', '2021_01_ethnicity_cpa_QA', '2021_01_ethnicity_jurisdiction_QA', '2021_01_ethnicity_region_QA', '2021_01_households_cpa_QA', '2021_01_households_jurisdiction_QA', '2021_01_households_region_QA', '2021_01_household_income_cpa_QA', '2021_01_household_income_jurisdiction_QA', '2021_01_household_income_region_QA'])

In [3]:
# This is the 2020_06 Data
# specifying the path to csv files
path = "C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-56 Estimates 2021 QC/Data/2020_06/Raw Data"
  
# csv files in the path
files = glob.glob(path + "/*.csv")

old = {}
for path in files:
    old[path.split('\\')[1][:-4]] = pd.read_csv(path)

In [7]:
old.keys()

dict_keys(['2020_06_age_cpa_QA', '2020_06_age_ethnicity_cpa_QA', '2020_06_age_ethnicity_jurisdiction_QA', '2020_06_age_ethnicity_region_QA', '2020_06_age_jurisdiction_QA', '2020_06_age_region_QA', '2020_06_age_sex_ethnicity_cpa_QA', '2020_06_age_sex_ethnicity_jurisdiction_QA', '2020_06_age_sex_ethnicity_region_QA', '2020_06_consolidated_cpa_QA', '2020_06_consolidated_jurisdiction_QA', '2020_06_consolidated_region_QA', '2020_06_ethnicity_cpa_QA', '2020_06_ethnicity_jurisdiction_QA', '2020_06_ethnicity_region_QA', '2020_06_household_income_cpa_QA', '2020_06_household_income_jurisdiction_QA', '2020_06_household_income_region_QA'])

# Internal Consistency

## Compare Totals
This is comparing the totals accross CPA, Jur, and Region to see if they are equivalent. 

In [4]:
def compare_totals(cpa_df, jur_df, region_df):
    """
    Aggregates sum of values by year in cpa, jurisdiction, and region level and compares them together to see if they match. CPA is removed as many areas calculated in the other georgraphy 
    """
    cpa_totals = cpa_df.groupby('yr_id').sum()
    jur_totals = jur_df.groupby('yr_id').sum()
    reg_totals = region_df
    
    non_matches_dict = {}
    jur_non_matches = []
    for col in cpa_totals.columns:
        if not np.isclose(jur_totals[col], cpa_totals[col]).all():
            jur_non_matches.append(col)
                        
    if len(jur_non_matches) == len(cpa_totals.columns):
        non_matches_dict['jurisdiction'] = 'all columns did not match.'
    elif len(jur_non_matches) == 0:
        non_matches_dict['jurisdiction'] = 'all columns matched.'
    else:
        if len(jur_non_matches) > 10:
            non_matches_dict['jurisdiction'] = f"{len(jur_non_matches)} columns did not match out of {len(cpa_totals.columns)} columns."
        else:
            non_matches_dict['jurisdiction'] = ', '.join(jur_non_matches) + ' columns did not match.'
    
    reg_non_matches = []
    for col in cpa_totals.columns:
        if not np.isclose(reg_totals[col], cpa_totals[col]).all():
            reg_non_matches.append(col)
    if len(reg_non_matches) == len(cpa_totals.columns):
        non_matches_dict['region'] = 'all columns did not match.'
    elif len(reg_non_matches) == 0:
        non_matches_dict['region'] = 'all columns matched.'
    else:
        if len(reg_non_matches) > 10:
            non_matches_dict['region'] = f"{len(reg_non_matches)} columns did not match out of {len(cpa_totals.columns)}."
        else:
            non_matches_dict['region'] = ', '.join(reg_non_matches) + ' columns did not match.'
    
    return non_matches_dict

In [5]:
compare_totals(new['2021_01_consolidated_cpa_QA'], new['2021_01_consolidated_jurisdiction_QA'], new['2021_01_consolidated_region_QA'])

{'jurisdiction': 'all columns matched.', 'region': 'all columns matched.'}

## Ethnicity population check 
Total Population for the region, jurisdiction, cpa matches the sum of all ethnicity group population 

In [6]:
def ethnicity_internal_consistency(cpa_df, jur_df, region_df):
    cpa_totals = cpa_df.groupby('yr_id').sum()
    jur_totals = jur_df.groupby('yr_id').sum()
    reg_totals = region_df


    results = {}

    for geography_data in [('CPA',cpa_totals), ('Jurisdcition',jur_totals), ('Region',reg_totals)]:
        
        geo_level = geography_data[0]
        df = geography_data[1]

        pop_test = df['Hispanic'] + df['Non-Hispanic, White'] + df['Non-Hispanic, Black'] + df['Non-Hispanic, American Indian or Alaska Native'] + df['Non-Hispanic, Asian'] + df['Non-Hispanic, Hawaiian or Pacific Islander'] + df['Non-Hispanic, Other'] + df['Non-Hispanic, Two or More Races']

        if sum(~(pop_test == df['Total Population'])) != 0:
            results[geo_level] = 'Ethnicity population does not sum up correctly'
        else:
            results[geo_level] = 'Pass'
    
    return results


In [7]:
ethnicity_internal_consistency(new['2021_01_consolidated_cpa_QA'], new['2021_01_consolidated_jurisdiction_QA'], new['2021_01_consolidated_region_QA'])

{'CPA': 'Pass', 'Jurisdcition': 'Pass', 'Region': 'Pass'}

## Age Group Comparison Check
Total Population for the region, jurisdiction, cpa matches the sum of all age-group population 

In [10]:
def age_group_internal_consistency(cpa_df, jur_df, region_df):
    cpa_totals = cpa_df.groupby('yr_id').sum()
    jur_totals = jur_df.groupby('yr_id').sum()
    reg_totals = region_df


    results = {}

    for geography_data in [('CPA',cpa_totals), ('Jurisdcition',jur_totals), ('Region',reg_totals)]:
        
        geo_level = geography_data[0]
        df = geography_data[1]

        pop_test = df['Under 5'] + df['5 to 9'] + df['10 to 14'] + df['15 to 17'] + df['18 and 19'] + df['20 to 24'] + df['25 to 29'] + df['30 to 34'] + df['35 to 39'] + df['40 to 44'] + df['45 to 49'] + df['50 to 54'] + df['55 to 59'] + df['60 and 61'] + df['62 to 64'] + df['65 to 69'] + df['70 to 74'] + df['75 to 79'] + df['80 to 84'] + df['85 and Older']

        if sum(~(pop_test == df['Total Population'])) != 0:
            results[geo_level] = 'Ethnicity population does not sum up correctly'
        else:
            results[geo_level] = 'Pass'
    
    return results

In [11]:
age_group_internal_consistency(new['2021_01_consolidated_cpa_QA'], new['2021_01_consolidated_jurisdiction_QA'], new['2021_01_consolidated_region_QA'])

{'CPA': 'Pass', 'Jurisdcition': 'Pass', 'Region': 'Pass'}

## Gender Population test 
Total Population for the region, jurisdiction, cpa matches the sum of male and female population from gender table 

In [12]:
def gender_internal_consistency(cpa_df, jur_df, region_df):
    cpa_totals = cpa_df.groupby('yr_id').sum()
    jur_totals = jur_df.groupby('yr_id').sum()
    reg_totals = region_df


    results = {}

    for geography_data in [('CPA',cpa_totals), ('Jurisdcition',jur_totals), ('Region',reg_totals)]:
        
        geo_level = geography_data[0]
        df = geography_data[1]

        pop_test = df['Female'] + df['Male']

        if sum(~(pop_test == df['Total Population'])) != 0:
            results[geo_level] = 'Ethnicity population does not sum up correctly'
        else:
            results[geo_level] = 'Pass'
    
    return results

In [13]:
gender_internal_consistency(new['2021_01_consolidated_cpa_QA'], new['2021_01_consolidated_jurisdiction_QA'], new['2021_01_consolidated_region_QA'])

{'CPA': 'Pass', 'Jurisdcition': 'Pass', 'Region': 'Pass'}

# Checking for Null Values
Code pulled from Prabina's code with slight modifications.

In [18]:
def spot_nulls(df):
    if (df.isna().sum() != 0).all():
        print('Null values present')
        return df.columns[df.isnull().any()].tolist() # returns the column names that have null vals
    else:
        print('No nulls Present')

In [19]:
spot_nulls(new['2021_01_consolidated_cpa_QA'])

No nulls Present


# Creating Diff Files

In [1]:
def diff_file(df1, df2, geo_level=None):
    '''
    This function creates a diff file of two dataframes with identical labels and identical shapes and creates a diff file of the two. 
    This does df1 minus df2
    
    '''
    if sum(~(df1.columns == df2.columns)) != 0:
        print('Columns are not identical, cannot preform diff')


    if geo_level == None:
        print('Please input a geography level')
    elif geo_level == 'cpa':
        new_df = (df1.set_index(['cpa', 'yr_id']) - df2.set_index(['cpa', 'yr_id'])).reset_index()    
    elif geo_level == 'jurisdiction':
        new_df = (df1.set_index(['jurisdiction', 'yr_id']) - df2.set_index(['jurisdiction', 'yr_id'])).reset_index()
    elif geo_level == 'region':
        new_df = (df1.set_index(['region', 'yr_id']) - df2.set_index(['region', 'yr_id'])).reset_index()

    
    return new_df

In [9]:
# Special for age_sex_ethnicity dataframe
def diff_file(df1, df2, geo_level=None):
    '''
    This function creates a diff file of two dataframes with identical labels and identical shapes and creates a diff file of the two. 
    This does df1 minus df2
    
    '''
    if sum(~(df1.columns == df2.columns)) != 0:
        print('Columns are not identical, cannot preform diff')


    if geo_level == None:
        print('Please input a geography level')
    elif geo_level == 'cpa':
        new_df = (df1.set_index(['cpa', 'yr_id', 'name', 'sex']) - df2.set_index(['cpa', 'yr_id', 'name', 'sex'])).reset_index()    
    elif geo_level == 'jurisdiction':
        new_df = (df1.set_index(['jurisdiction', 'yr_id', 'name', 'sex']) - df2.set_index(['jurisdiction', 'yr_id', 'name', 'sex'])).reset_index()
    elif geo_level == 'region':
        new_df = (df1.set_index(['region', 'yr_id', 'name', 'sex']) - df2.set_index(['region', 'yr_id', 'name', 'sex'])).reset_index()

    
    return new_df

## LUZ Diff File Creation

In [5]:
mgra_13_luz = pd.read_csv('C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-58 2019 Base Year Forecast Output QC/data/MGRA13 Updated Data/mgra13_update_luz_ind_QA.csv')
mgra_15_luz = pd.read_csv('C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-58 2019 Base Year Forecast Output QC/data/MGRA15 Data/mgra15_luz_ind_QA.csv')

In [7]:
a = list(mgra_13_luz.columns)
b = list(mgra_15_luz.columns)

# Find the columns that eist in both lists 
list(set(a) & set(b))

['emp_utilities_office',
 'beachactive',
 '75 to 79',
 'emp_public_ed',
 'pseudomsa',
 'emp_const_bldg_office',
 'totint',
 'emp_state_local_gov_ent',
 'othercollegeenroll',
 'dparkcost',
 'zip09',
 'emp_religious',
 'empdenbin',
 '5 to 9',
 'hch_dist',
 'emp_personal_svcs_office',
 'vacancy',
 'Female',
 'Two or More',
 'i3',
 'emp_utilities_prod',
 'Black',
 'emp_st_lcl_gov_accts',
 'i4',
 '65 to 69',
 'district27',
 'empden',
 '85 and Older',
 'emp_state_local_gov_blue',
 'hh_sf',
 'duden',
 '15 to 17',
 'pop',
 'economyroom',
 'acres',
 'Asian',
 'American Indian',
 'emp_fed_gov_accts',
 'luxuryroom',
 'i9',
 'emp_fed_non_mil',
 'hs_mf',
 'emp_const_bldg_prod',
 '45 to 49',
 'hs_sf',
 'land_acres',
 '80 to 84',
 'emp_pvt_ed_k12',
 'emp_restaurant_bar',
 'hparkcost',
 'midpriceroom',
 'effective_acres',
 'hstallssam',
 'elem_population',
 'emp_own_occ_dwell_mgmt',
 '25 to 29',
 'enrollgrade9to12',
 'truckregiontype',
 'collegeenroll',
 'emp_fed_mil',
 'emp_trans',
 'enrollgradekto8'

In [14]:
# Create identical columns
mgra_13_luz_clean = mgra_13_luz[list(set(a) & set(b))].set_index('luz_id')
mgra_15_luz_clean = mgra_15_luz[list(set(a) & set(b))].set_index('luz_id')

In [18]:
(mgra_13_luz_clean - mgra_15_luz_clean).to_csv('C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-58 2019 Base Year Forecast Output QC/data/DIFF/mgra_13_luz_minus_mgra_15_luz.csv')

In [19]:
# Finding just the columns with 'emp' 
new_df = (mgra_13_luz_clean - mgra_15_luz_clean)

new_df[[x for x in mgra_15_luz if x[0:3] == 'emp']].to_csv('C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-58 2019 Base Year Forecast Output QC/data/DIFF/mgra_13_luz_minus_mgra_15_luz_emp_output.csv')

## Age Sex Ethnicity/DOF Proportion Differences 

### Column-Wise Difference

In [60]:
# Column-wise 2021_01 Age-sex-ethnicity Proportion Data
data_dof = pd.read_csv('C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-56 Estimates 2021 QC/Data/2021_01/Proportion Checks/age_sex_ethnicity_col_wise_sums.csv')

# Column-wise 2021 age-sex-ethnicity DOF data
col_dof = pd.read_csv('C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-56 Estimates 2021 QC/Data/2021_01/DOF_Data/DOF_age_sex_ethnicity_col_wise_sums.csv')

In [61]:
# Filter for like columns between the two 
data_dof = data_dof[['region', 'yr_id', 'name', 'sex', 'Hispanic', 'Non-Hispanic, White',
       'Non-Hispanic, Asian', 'Non-Hispanic, Hawaiian or Pacific Islander',
       'Non-Hispanic, American Indian or Alaska Native',
       'Non-Hispanic, Two or More Races', 'Non-Hispanic, Black']]# removed "non-hispanic other"

col_dof = col_dof[['region', 'yr_id', 'name', 'sex', 'Hispanic', 'Non-Hispanic, White',
       'Non-Hispanic, Asian', 'Non-Hispanic, Hawaiian or Pacific Islander',
       'Non-Hispanic, American Indian or Alaska Native',
       'Non-Hispanic, Two or More Races', 'Non-Hispanic, Black']]



In [66]:
# Create the diff table
table = diff_file(data_dof, col_dof, geo_level='region')
table

,region,yr_id,name,sex,Hispanic,"Non-Hispanic, White","Non-Hispanic, Asian","Non-Hispanic, Hawaiian or Pacific Islander","Non-Hispanic, American Indian or Alaska Native","Non-Hispanic, Two or More Races","Non-Hispanic, Black"
0,San Diego,2010,10 to 14,F,0.021561,0.002368,-0.009153,0.068970,-0.035449,-0.078479,-0.015411
1,San Diego,2010,10 to 14,M,0.007028,-0.004973,-0.014111,0.059427,-0.162044,-0.124233,0.004538
2,San Diego,2010,15 to 17,F,0.295709,-0.124931,-0.301220,-0.169267,-0.086792,0.353501,0.179717
3,San Diego,2010,15 to 17,M,0.328962,-0.166806,-0.247008,-0.099522,0.027308,0.383080,0.180005
4,San Diego,2010,18 and 19,F,0.152597,-0.064948,0.038031,0.094088,-0.212420,-0.085410,-0.052656
...,...,...,...,...,...,...,...,...,...,...,...
475,San Diego,2021,80 to 84,M,-0.002961,-0.017871,-0.026333,0.054404,-0.027302,0.003168,0.008439
476,San Diego,2021,85 and Older,F,-0.016035,-0.013002,-0.034302,-0.024285,0.079365,0.051725,-0.016126
477,San Diego,2021,85 and Older,M,-0.007469,-0.014931,-0.010849,-0.021384,0.049011,0.028539,-0.012501
478,San Diego,2021,Under 5,F,0.131091,0.042229,0.055036,0.032791,-0.028686,-0.006429,0.036967


In [67]:
# with pd.ExcelWriter("C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-56 Estimates 2021 QC/Data/2021_01/Diff Files/DOF_diff_col_wise.xlsx") as writer:
   
#     # use to_excel function and specify the sheet_name and index
#     # to store the dataframe in specified sheet
#     data_dof.to_excel(writer, sheet_name="DOF", index=False)
#     col_dof.to_excel(writer, sheet_name="2021_01", index=False)
#     table.to_excel(writer, sheet_name="Diff - DOF minus 2021_01", index=False)

### Row-wise Difference

In [84]:
# Row-wise 2021_01 Age-sex-ethnicity Proportion Data
data_dof = pd.read_csv('C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-56 Estimates 2021 QC/Data/2021_01/Proportion Checks/age_sex_ethnicity_row_wise_sums.csv')

# Row-wise 2021 age-sex-ethnicity DOF data
col_dof = pd.read_csv('C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-56 Estimates 2021 QC/Data/2021_01/DOF_Data/DOF_age_sex_ethnicity_row_wise_sums.csv')

In [87]:
data_dof = data_dof[['region', 'yr_id', 'name', 'sex', 'Hispanic', 'Non-Hispanic, White',
       'Non-Hispanic, Asian', 'Non-Hispanic, Hawaiian or Pacific Islander',
       'Non-Hispanic, American Indian or Alaska Native',
       'Non-Hispanic, Two or More Races', 'Non-Hispanic, Black']] # removed "non-hispanic other"

data_dof['sex'] = col_dof['sex'] # Equating the two sex columns because one goes by "M/F" while the other goes by "Male/Female"

In [88]:
col_dof = col_dof[['region', 'yr_id', 'name', 'sex', 'Hispanic', 'Non-Hispanic, White',
       'Non-Hispanic, Asian', 'Non-Hispanic, Hawaiian or Pacific Islander',
       'Non-Hispanic, American Indian or Alaska Native',
       'Non-Hispanic, Two or More Races', 'Non-Hispanic, Black']]

In [89]:
table = diff_file(data_dof, col_dof, geo_level='region')
table

,region,yr_id,name,sex,Hispanic,"Non-Hispanic, White","Non-Hispanic, Asian","Non-Hispanic, Hawaiian or Pacific Islander","Non-Hispanic, American Indian or Alaska Native","Non-Hispanic, Two or More Races","Non-Hispanic, Black"
0,San Diego,2010,10 to 14,F,0.003099,-0.172497,-0.130148,0.004132,-0.010329,0.127049,-0.083666
1,San Diego,2010,10 to 14,M,-0.006684,-0.177139,-0.118301,0.003281,-0.026926,0.099129,-0.038318
2,San Diego,2010,15 to 17,F,4.222597,-3.591888,-1.801953,-0.047799,-0.030094,0.668915,0.327214
3,San Diego,2010,15 to 17,M,4.447925,-4.271212,-1.398805,-0.030476,-0.003211,0.673394,0.299005
4,San Diego,2010,18 and 19,F,2.831824,-2.734724,0.067719,0.018460,-0.077871,-0.053076,-0.280565
...,...,...,...,...,...,...,...,...,...,...,...
475,San Diego,2021,80 to 84,M,0.094490,-0.134590,-0.242852,0.032685,-0.011220,0.068619,0.071840
476,San Diego,2021,85 and Older,F,-0.244522,0.225101,-0.213407,-0.006775,0.030432,0.182179,-0.052114
477,San Diego,2021,85 and Older,M,-0.134264,-0.010863,-0.065229,-0.009293,0.032999,0.176054,-0.062468
478,San Diego,2021,Under 5,F,0.425793,-0.462668,-0.092663,-0.007021,-0.019142,0.031116,-0.112649


In [90]:
# with pd.ExcelWriter("C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-56 Estimates 2021 QC/Data/2021_01/Diff Files/DOF_diff_row_wise.xlsx") as writer:
   
#     # use to_excel function and specify the sheet_name and index
#     # to store the dataframe in specified sheet
#     data_dof.to_excel(writer, sheet_name="DOF", index=False)
#     col_dof.to_excel(writer, sheet_name="2021_01", index=False)
#     table.to_excel(writer, sheet_name="Diff - DOF minus 2021_01", index=False)

# Year on Year output file

In [37]:
def yoy_pct_diff(df, geo_level=None):
    '''
    This finds the percent change by year
    df must be already indexed
    
    '''
    if geo_level == None:
        print('Please input a valid geography level')
    elif geo_level == 'region':
        final = df.pct_change()*100
    elif geo_level == 'cpa':
        final = pd.DataFrame()
        for geography in list(set(df.index.get_level_values(0))):
            final = final.append(df.loc[[geography]].pct_change()*100)
    elif geo_level == 'jurisdiction':
        final = pd.DataFrame()
        for geography in list(set(df.index.get_level_values(0))):
            final = final.append(df.loc[[geography]].pct_change()*100)
    return final

In [49]:
# yoy_pct_diff(old['2020_06_consolidated_region_QA'].set_index(['region', 'yr_id']), geo_level='region').to_csv('C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-56 Estimates 2021 QC/Data/2020_06/yoy_pct_change/consolidated_region_change.csv')

# Proportion Check 
1. household income

2. age

3. gender (later)

4. ethnicity

5. ethnicity by age

6. ethnicity by age by gender


## Row-wise proportion checks
This sums up a particular row then each cell is divided by that row. This tells the percentage a particular cell makes up for a particular row. 

In [171]:
# Depending on the dataset we are using the groupby function will have to change. 
df = new['2021_01_age_sex_ethnicity_region_QA'].groupby(['region', 'yr_id', 'name', 'sex']).sum()
sum_vals = list(df.sum(axis=1))
row_sum_percentage = df.div(sum_vals, axis=0)*100
row_sum_percentage

Hispanic  Non-Hispanic, White  \
region    yr_id name         sex                                      
San Diego 2010  10 to 14     Female  45.614742            34.159669   
                             Male    45.021246            34.568168   
                15 to 17     Female  45.171754            35.049791   
                             Male    45.199120            34.790932   
                18 and 19    Female  42.059430            36.785659   
...                                        ...                  ...   
          2021  80 to 84     Male    15.476551            67.348714   
                85 and Older Female  16.367428            67.420955   
                             Male    14.746712            70.254912   
                Under 5      Female  41.236295            39.099133   
                             Male    40.848106            39.466706   

                                     Non-Hispanic, Asian  \
region    yr_id name         sex                           
San Diego 2010  10 to 14     Female             8.918224   
                             Male               9.142027   
                15 to 17     Female             8.620130   
                             Male               8.667209   
                18 and 19    Female            10.831173   
...                                                  ...   
          2021  80 to 84     Male              11.490166   
                85 and Older Female            11.757797   
                             Male              10.070628   
                Under 5      Female             8.916876   
                             Male               8.775014   

                                     Non-Hispanic, Hawaiian or Pacific Islander  \
region    yr_id name         sex                                                  
San Diego 2010  10 to 14     Female                                    0.397674   
                             Male                                      0.425895   
                15 to 17     Female                                    0.437899   
                             Male                                      0.465768   
                18 and 19    Female                                    0.452037   
...                                                                         ...   
          2021  80 to 84     Male                                      0.431165   
                85 and Older Female                                    0.335595   
                             Male                                      0.434324   
                Under 5      Female                                    0.344504   
                             Male                                      0.355529   

                                     Non-Hispanic, American Indian or Alaska Native  \
region    yr_id name         sex                                                      
San Diego 2010  10 to 14     Female                                        0.411102   
                             Male                                          0.464167   
                15 to 17     Female                                        0.471958   
                             Male                                          0.473305   
                18 and 19    Female                                        0.478628   
...                                                                             ...   
          2021  80 to 84     Male                                          0.446293   
                85 and Older Female                                        0.321212   
                             Male                                          0.381555   
                Under 5      Female                                        0.438366   
                             Male                                          0.514425   

                                     Non-Hispanic, Other  \
region    yr_id name         sex                           
S

In [172]:
row_sum_percentage.to_csv('C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-56 Estimates 2021 QC/Data/2021_01/Proportion Checks/age_sex_ethnicity_row_wise_sums.csv')

## Column-wise proportion checks
This takes a particular row and sums it up. Then for an individual cell it is divided for that column. This is done uniquely for each year. 

### Ethnicicty by Age Column-wise Proportion

In [ ]:
final = pd.DataFrame()

for year in list(set(new['2021_01_age_ethnicity_region_QA'].groupby(['region', 'yr_id', 'name']).sum().index.get_level_values(1))):

    df = new['2021_01_age_ethnicity_region_QA'].groupby(['region', 'yr_id', 'name']).sum().loc[("San Diego", year)]


    sum_vals = list(df.sum(axis=0))
    col_sum_percentage = df.div(sum_vals, axis=1)*100

    df = col_sum_percentage


    df['region'] = 'San Diego'
    df['yr_id'] = year


    df = df.reset_index()
    final = final.append(df)


# final.groupby(['region', 'yr_id', 'name']).sum().to_csv('C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-56 Estimates 2021 QC/Data/2021_01/Proportion Checks/ethnicity_by_age_col_wise_sums.csv')

### Age-Sex-Ethnicity Column-wise Proportion 

In [176]:
# Age Sex Ethnicity Code 

final = pd.DataFrame()


for year in list(set(new['2021_01_age_sex_ethnicity_region_QA'].groupby(['region', 'yr_id', 'name', 'sex']).sum().index.get_level_values(1))):

    df = new['2021_01_age_sex_ethnicity_region_QA'].groupby(['region', 'yr_id', 'name', 'sex']).sum().loc[("San Diego", year)]


    sum_vals = list(df.sum(axis=0))
    col_sum_percentage = df.div(sum_vals, axis=1)*100

    df = col_sum_percentage


    df['region'] = 'San Diego'
    df['yr_id'] = year


    df = df.reset_index()
    final = final.append(df)


final.groupby(['region', 'yr_id', 'name', 'sex']).sum()


final.groupby(['region', 'yr_id', 'name', 'sex']).sum().to_csv('C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-56 Estimates 2021 QC/Data/2021_01/Proportion Checks/age_sex_ethnicity_col_wise_sums.csv')

In [130]:
# col_sum_percentage.to_csv('C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-56 Estimates 2021 QC/Data/2021_01/Proportion Checks/ethnicity_by_age_col_wise_sums.csv')

## DOF Proportion Checks 

In [77]:
# Download the data
dof_data = pd.read_csv('C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-56 Estimates 2021 QC/Data/2021_01/DOF_Data/DOF_2021_07_14_age_sex_ethnicity_region_QA.csv')

In [78]:
# Checking the years that are present in a table from database
list(set(new['2021_01_age_sex_ethnicity_region_QA'].groupby(['region', 'yr_id', 'name', 'sex']).sum().index.get_level_values(1)))

[2016, 2017, 2018, 2019, 2020, 2021, 2010, 2011, 2012, 2013, 2014, 2015]

In [81]:
# Fltering DOF data for years that exist in our estimates data 
dof_data = dof_data[dof_data['yr_id'].isin(list(set(new['2021_01_age_sex_ethnicity_region_QA'].groupby(['region', 'yr_id', 'name', 'sex']).sum().index.get_level_values(1))))]

In [82]:
# Row-wise proportion checks
df = dof_data.groupby(['region', 'yr_id', 'name', 'sex']).sum()
sum_vals = list(df.sum(axis=1))
row_sum_percentage = df.div(sum_vals, axis=0)*100
row_sum_percentage

Hispanic  \
region    yr_id name         sex              
San Diego 2010  10 to 14     F    45.611643   
                             M    45.027930   
                15 to 17     F    40.949157   
                             M    40.751195   
                18 and 19    F    39.227606   
...                                     ...   
          2021  80 to 84     M    15.382061   
                85 and Older F    16.611950   
                             M    14.880976   
                Under 5      F    40.810502   
                             M    40.443574   

                                  Non-Hispanic, American Indian or Alaska Native  \
region    yr_id name         sex                                                   
San Diego 2010  10 to 14     F                                          0.421431   
                             M                                          0.491093   
                15 to 17     F                                          0.502052   
                             M                                          0.476516   
                18 and 19    F                                          0.556499   
...                                                                          ...   
          2021  80 to 84     M                                          0.457514   
                85 and Older F                                          0.290779   
                             M                                          0.348556   
                Under 5      F                                          0.457507   
                             M                                          0.524893   

                                  Non-Hispanic, Asian  Non-Hispanic, Black  \
region    yr_id name         sex                                             
San Diego 2010  10 to 14     F               9.048372             5.017921   
                             M               9.260328             5.016582   
                15 to 17     F              10.422084             4.960016   
                             M              10.066014             5.287199   
                18 and 19    F              10.763454             5.354908   
...                                               ...                  ...   
          2021  80 to 84     M              11.733018             3.468251   
                85 and Older F              11.971203             2.724885   
                             M              10.135858             3.045906   
                Under 5      F               9.009538             4.916106   
                             M               8.896334             4.939757   

                                  Non-Hispanic, Hawaiian or Pacific Islander  \
region    yr_id name         sex                                               
San Diego 2010  10 to 14     F                                      0.393542   
                             M                                      0.422614   
                15 to 17     F                                      0.485699   
                             M                                      0.496244   
                18 and 19    F                                      0.433578   
...                                                                      ...   
          2021  80 to 84     M                                      0.398480   
                85 and Older F                                      0.342369   
                             M                                      0.443617   
                Under 5      F                                      0.351525   
                             M                                      0.358875   

                                  Non-Hispanic, Two or More Races  \
region    yr_id name         sex                                    
San Diego 2010  10 to 14     F                           5.174925   
                             M                           5.036147  

In [83]:
# row_sum_percentage.to_csv('C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-56 Estimates 2021 QC/Data/2021_01/DOF_Data/DOF_age_sex_ethnicity_row_wise_sums.csv')

In [11]:
# DOF column-wise checks (this code is nearly identical to the age-sex-ethnicity code)

final = pd.DataFrame()

for year in list(set(new['2021_01_age_sex_ethnicity_region_QA'].groupby(['region', 'yr_id', 'name', 'sex']).sum().index.get_level_values(1))):

    df = dof_data.groupby(['region', 'yr_id', 'name', 'sex']).sum().loc[("San Diego", year)]


    sum_vals = list(df.sum(axis=0))
    col_sum_percentage = df.div(sum_vals, axis=1)*100

    df = col_sum_percentage


    df['region'] = 'San Diego'
    df['yr_id'] = year


    df = df.reset_index()
    final = final.append(df)


final.groupby(['region', 'yr_id', 'name', 'sex']).sum()


# final.groupby(['region', 'yr_id', 'name', 'sex']).sum().to_csv('C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-56 Estimates 2021 QC/Data/2021_01/DOF_Data/DOF_age_sex_ethnicity_col_wise_sums.csv')

# Department of Finance Checks 
There's 2 DOF sources that the estimates control to. The 2021 vintage DOF Population and Housing Estimates E5 that's in [socioec_data].[ca_dof].[population_housing_estimates]. This gives household, group quarter data jurisdiction level. Then there's the 7.14.2021 DOF Projections that gives single year age group and ethnicity characteristics. It's also in the socioec_data database I just don't have the name of the table at the moment since I can't remote access my computer since the power outage.

### 2021 Vintage DOF Population and Housing Estimates E5
[socioec_data].[ca_dof].[population_housing_estimates]


- Which year do I compare the DOF numbers to? The base year?

In [21]:
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                    'Server=DDAMWSQL16.sandag.org;'
                    'Database=socioec_data;'
                    'Trusted_Connection=yes;')

In [73]:
dof_pop_query = '''SELECT *
  FROM [socioec_data].[ca_dof].[population_housing_estimates]
  WHERE area_name = 'San Diego' AND summary_type='Total' AND area_type = 'county' AND vintage_yr = 2021;'''

dof_pop =  pd.read_sql_query(dof_pop_query, conn) #example df

dof_pop

,area_type,area_name,summary_type,county_name,est_yr,est_md,total_pop,household_pop,group_quarters,total_hu,single,single_detached,single_attached,multiple,two_to_four,five_plus,mobile_homes,occupied,unoccupied,vintage_yr
0,County,San Diego,Total,San Diego,2017,JAN_1,3303366,3197067,106299,1201517,721664,616128,105536,433837,86281,347556,46016,1129464,72053,2021
1,County,San Diego,Total,San Diego,2010,JAN_1,3095313,2991515,103798,1164028,707604,603441,104163,410658,84621,326037,45766,1086113,77915,2021
2,County,San Diego,Total,San Diego,2014,JAN_1,3232762,3127529,105233,1180875,715522,610783,104739,419449,85494,333955,45904,1107562,73313,2021
3,County,San Diego,Total,San Diego,2020,JAN_1,3331279,3218869,112410,1226879,729293,622861,106432,451206,87814,363392,46380,1153981,72898,2021
4,County,San Diego,Total,San Diego,2011,JAN_1,3127603,3025703,101900,1166858,709313,605022,104291,411748,84620,327128,45797,1089420,77438,2021
5,County,San Diego,Total,San Diego,2018,JAN_1,3321118,3210345,110773,1210138,723430,617674,105756,440623,86682,353941,46085,1137791,72347,2021
6,County,San Diego,Total,San Diego,2015,JAN_1,3264706,3156628,108078,1188445,717461,612517,104944,425048,85792,339256,45936,1114909,73536,2021
7,County,San Diego,Total,San Diego,2021,JAN_1,3315404,3219058,96346,1234087,730771,624045,106726,456850,89426,367424,46466,1160772,73315,2021
8,County,San Diego,Total,San Diego,2012,JAN_1,3161808,3058316,103492,1170267,711359,606978,104381,413069,84884,328185,45839,1093511,76756,2021
9,County,San Diego,Total,San Diego,2016,JAN_1,3283009,3175539,107470,1193395,719600,614416,105184,427835,85921,341914,45960,1121242,72153,2021


### Difference Between DOF Data And Our Estimates
Here I will be taking the data that we get from DOF and finding the difference between DOF data and our estimates

In [85]:
DOF_1_check = pd.DataFrame()

temp_df = pd.DataFrame()

for year in list(new['2021_01_consolidated_region_QA']['yr_id']):
    # Data Setup 
    dof_df = dof_pop[dof_pop['est_yr'] == year]
    estimates_dof = new['2021_01_consolidated_region_QA'][new['2021_01_consolidated_region_QA']['yr_id'] == year]

    # Population Commparison
    dof_total_population_value =  int(dof_df['total_pop']) 
    estimates_total_population_value = int(estimates_dof['Total Population'])
    if estimates_total_population_value == 0:
        temp_df['Household Population Diff'] = [f"Zero in denominater, actual diff is: {dof_total_population_value-estimates_total_population_value}"]
    else:
        temp_df['Total Population Diff'] = [dof_total_population_value-estimates_total_population_value]

    # Household Population Comparison 
    dof_household_population_value =  int(dof_df['household_pop']) 
    estimates_household_population_value = int(estimates_dof['Household Population'])
    if estimates_household_population_value == 0:
        temp_df['Household Population Diff'] = [f"Zero in denominater, actual diff is: {dof_household_population_value-estimates_household_population_value}"]
    else: 
        temp_df['Household Population Diff'] = [dof_household_population_value-estimates_household_population_value]

    # Group Quarters Comparison 
    dof_group_quarters_value =  int(dof_df['group_quarters']) 
    estimates_group_quarters_value = int(estimates_dof['Group Quarters - Military']) + int(estimates_dof['Group Quarters - College']) + int(estimates_dof['Group Quarters - Other'])
    if estimates_group_quarters_value == 0:
        temp_df['Group Quarters Diff'] = [f"Zero in denominater, actual diff is: {dof_group_quarters_value-estimates_group_quarters_value}"]
    else: 
        temp_df['Group Quarters Diff'] = [dof_group_quarters_value-estimates_group_quarters_value]

    # # Single Family Detached Comparison 
    # dof_single_detached_value =  int(dof_df['single_detached']) 
    # estimates_single_detached_value = int(estimates_dof['Single Family - Detached'])
    # temp_df['Single Detached Diff'] = [dof_single_detached_value-estimates_single_detached_value]

    # # Single Family Attchaded Comparison 
    # dof_single_attached_value =  int(dof_df['single_attached']) 
    # estimates_single_attached_value = int(estimates_dof['Single-family Attached'])
    # if estimates_single_attached_value == 0:
    #     temp_df['Single Attached Diff'] = [dof_single_attached_value-estimates_single_attached_value]
    # else:
    #     temp_df['Single Attached Diff'] = [dof_single_attached_value-estimates_single_attached_value]
    
    # Total Housing Units Comparison 
    dof_total_housing_value =  int(dof_df['total_hu']) 
    estimates_total_housing_value = int(estimates_dof['units'])
    temp_df['Total Housing Diff'] = [dof_total_housing_value-estimates_total_housing_value]

    # Total Vacancy Comparison 
    dof_vacancy_value =  int(dof_df['unoccupied']) 
    estimates_vacancy_value = int(estimates_dof['vacancy'])
    temp_df['Vacancy Diff'] = [dof_vacancy_value-estimates_vacancy_value]


    # Setup the Year 
    temp_df['Year'] = year

    DOF_1_check = DOF_1_check.append(temp_df)
DOF_1_check.set_index('Year')

,Total Population Diff,Household Population Diff,Group Quarters Diff,Total Housing Diff,Vacancy Diff
Year,,,,,
2010,-1,-1833,1832,9026,9779
2011,-1,-1,0,8434,8435
2012,-1,-1,0,7935,7936
2013,-1,-1,0,9363,9364
2014,-1,-1,0,8841,8842
2015,-1,-1,0,9588,9589
2016,-1,-1,0,10412,10413
2017,-1,-1,0,10962,10963
2018,0,0,0,12429,12429


### Second DOF Data

In [68]:
dof_eth_query = '''SELECT [county_fips_code]
      ,[fiscal_yr]
      ,[race_code]
      ,[sex]
      ,[age]
      ,[population]
  FROM [socioec_data].[ca_dof].[population_proj_2021_07_14]
  WHERE county_fips_code = '06073';'''

dof_eth =  pd.read_sql_query(dof_eth_query, conn) #example df
dof_eth

,county_fips_code,fiscal_yr,race_code,sex,age,population
0,6073,2029,6,M,109,0
1,6073,2029,6,M,110,0
2,6073,2029,7,F,0,6838
3,6073,2029,7,F,1,6687
4,6073,2029,7,F,2,6991
...,...,...,...,...,...,...
79249,6073,2060,7,M,106,7
79250,6073,2060,7,M,107,1
79251,6073,2060,7,M,108,1
79252,6073,2060,7,M,109,1
